In [1]:
from m5_pipeline import M5

In [2]:
from language import Language

In [3]:
configuration=M5('localhost:8081', 'HCHS', 1, 'anamnese')

In [4]:
list_elements=[]

In [5]:
for i, j in configuration.loader(Language.GERMAN):
    print(i)
    list_elements.append(i) 
    print(j)

hch_anam0001
Wird die Anamnese durchgeführt?
hch_anam0002
Warum wird die Anamnese nicht durchgeführt?
hch_anam0003
Alter zwischen 45 und 74 Jahren
hch_anam0004
Einwohner/in der Stadt Hamburg


In [6]:
translated_variables=['Is the medical history taken?', 'Why is the anamnesis not performed?',
'Age between 45 and 74 years',
'Resident of the city of Hamburg']

In [7]:
configuration.translation_uploader(list_elements, translated_variables, Language.ENGLISH)

INFO:root:hch_anam0001 updated
INFO:root:hch_anam0002 updated
INFO:root:hch_anam0003 updated
INFO:root:hch_anam0004 updated


Concept loading

In [8]:
import pandas as pd

In [13]:
df=pd.read_excel('C://Users/admin/OneDrive/Desktop/work_project/de.uke.iam.automapping/src/automapping/2022_11_16_hchs_description_snomed.xlsx')

In [16]:
df_small=df[50:55]

In [17]:
df_small

,Unnamed: 0,SourceName,targetConceptName,targetConceptID,targetDomainID,MatchScore
50,50,adjunct pleura,adjunct - intent,4227509,Observation,0.707481
51,51,adjunct pleura,pleura,4150673,Synonym,0.575821
52,52,adjunct pleura,poisoning by pharmaceutical adjunct,4186783,Condition,0.569787
53,53,adjunct pleura,sedation with analgesic adjunct,4083088,Procedure,0.534084
54,54,adjunct pleura,pleura finding,4116786,Condition,0.520927


In [24]:
df_concept=pd.read_csv('C://Users/admin/OneDrive/Desktop/work_project/de.uke.iam.automapping/src/automapping/CONCEPT.csv', on_bad_lines='skip', delimiter='\t')

C:\Users\admin\AppData\Local\Temp\ipykernel_20160\897399409.py:1: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_concept=pd.read_csv('C://Users/admin/OneDrive/Desktop/work_project/de.uke.iam.automapping/src/automapping/CONCEPT.csv', on_bad_lines='skip', delimiter='\t')


In [27]:
df_concept = df_concept[
            (df_concept["vocabulary_id"].isin(['SNOMED']))
            & (df_concept["standard_concept"] == "S")
        ]

In [35]:
df_concept[df_concept['concept_id']==4227509].iloc[0]['concept']

421974008

In [38]:
df_small=df_small.merge(df_concept, left_on='targetConceptID', right_on='concept_id', how='left')

In [39]:
df_small

,Unnamed: 0,SourceName,targetConceptName,targetConceptID,targetDomainID,MatchScore,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,50,adjunct pleura,adjunct - intent,4227509,Observation,0.707481,4227509,Adjunct - intent,Observation,SNOMED,Qualifier Value,S,421974008,20060731,20991231,NaN
1,51,adjunct pleura,pleura,4150673,Synonym,0.575821,4150673,Pleural structure,Spec Anatomic Site,SNOMED,Body Structure,S,3120008,20020131,20991231,NaN
2,52,adjunct pleura,poisoning by pharmaceutical adjunct,4186783,Condition,0.569787,4186783,Poisoning by pharmaceutical adjunct,Condition,SNOMED,Clinical Finding,S,46571000,20020131,20991231,NaN
3,53,adjunct pleura,sedation with analgesic adjunct,4083088,Procedure,0.534084,4083088,Sedation with analgesic adjunct,Procedure,SNOMED,Procedure,S,241712003,20020131,20991231,NaN
4,54,adjunct pleura,pleura finding,4116786,Condition,0.520927,4116786,Pleura finding,Condition,SNOMED,Clinical Finding,S,301256004,20020131,20991231,NaN


In [40]:
df_small=df_small[['vocabulary_id', 'concept_code', 'MatchScore']]

In [44]:
df_voc=pd.read_csv('C://Users/admin/OneDrive/Desktop/work_project/de.uke.iam.automapping/src/automapping/VOCABULARY (1).csv',on_bad_lines='skip', delimiter='\t')

In [46]:
df_voc[df_voc['vocabulary_id']=='SNOMED'].iloc[0]['vocabulary_version']

'2020-07-31 SNOMED CT International Edition; 2020-09-01 SNOMED CT US Edition; 2020-10-28 SNOMED CT UK Edition'

In [47]:
df_small['Version']=df_voc[df_voc['vocabulary_id']=='SNOMED'].iloc[0]['vocabulary_version']

C:\Users\admin\AppData\Local\Temp\ipykernel_20160\2334258289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['Version']=df_voc[df_voc['vocabulary_id']=='SNOMED'].iloc[0]['vocabulary_version']


In [48]:
df_small

,vocabulary_id,concept_code,MatchScore,Version
0,SNOMED,421974008,0.707481,2020-07-31 SNOMED CT International Edition; 20...
1,SNOMED,3120008,0.575821,2020-07-31 SNOMED CT International Edition; 20...
2,SNOMED,46571000,0.569787,2020-07-31 SNOMED CT International Edition; 20...
3,SNOMED,241712003,0.534084,2020-07-31 SNOMED CT International Edition; 20...
4,SNOMED,301256004,0.520927,2020-07-31 SNOMED CT International Edition; 20...


In [49]:
import requests

In [50]:
body = requests.get('http://localhost:8081/m5.rest/api/HCHS/1/anamnese/hch_anam0003', timeout=10).json()

In [65]:
body['concepts']=[]
for i, data in df_small.iterrows():
    body['concepts'].append(
    {
        "conceptType": data['vocabulary_id'],
        "conceptCode": data['concept_code'],
        "conceptVersion": data['Version'],
        "conceptScore": data["MatchScore"],
        "verified": 'false'
    }
)

In [66]:
body

{'name': 'hch_anam0003',
 'labels': [{'language': 'EN',
   'value': 'Age between 45 and 74 years',
   'type': 'PREFERED'},
  {'language': 'DE',
   'value': 'Alter zwischen 45 und 74 Jahren',
   'type': 'PREFERED'}],
 'definitions': {'DE': 'Alter zwischen 45 und 74 Jahren',
  'EN': 'Age between 45 and 74 years'},
 'valueDomain': {'ValueDomain': 'EnumeratedValueDomain'},
 'dataType': 'CATALOG',
 '_links': {'self': {'href': 'http://localhost:8081/m5.rest/api/HCHS/1/anamnese/hch_anam0003'}},
 'concepts': [{'conceptType': 'SNOMED',
   'conceptCode': 421974008,
   'conceptVersion': '2020-07-31 SNOMED CT International Edition; 2020-09-01 SNOMED CT US Edition; 2020-10-28 SNOMED CT UK Edition',
   'conceptScore': 0.707480636634564,
   'verified': 'false'},
  {'conceptType': 'SNOMED',
   'conceptCode': 3120008,
   'conceptVersion': '2020-07-31 SNOMED CT International Edition; 2020-09-01 SNOMED CT US Edition; 2020-10-28 SNOMED CT UK Edition',
   'conceptScore': 0.5758211255842259,
   'verified': 

In [67]:
header = {"Content-type": "application/json", "Accept": "application/json"}

In [68]:
requests.patch('http://localhost:8081/m5.rest/api/HCHS/1/anamnese/hch_anam0003', json=body, headers=header, timeout=10)

<Response [200]>